In [1]:
library(dplyr)
library(tidyr)
library(tidyverse)
library(tidymodels)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ r

In [2]:
tennis_data <- read_csv("data/player_stats.csv")

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
tennis_data_deleted <-tennis_data |>rename(Current_Rank = "Current Rank",Best_rank ="Best Rank",Favorite_Surface="Favorite Surface",Turned_Pro="Turned Pro",Current_Elo_Rank="Current Elo Rank",Best_Elo_Rank="Best Elo Rank",Peak_Elo_Rating="Peak Elo Rating",Last_Appearance="Last Appearance",GOAT_Rank="GOAT Rank",Best_Season="Best Season")|>
select(- c(...1,Wikipedia, Height,Masters,Birthplace,Residence,Weight,Coach,Facebook,Twitter,Nicknames,"Davis Cups",Olympics,"Grand Slams","Web Site","Team Cups","Weeks at No. 1","Tour Finals"))

In [4]:
Separated_Age_Age_When <- separate(tennis_data_deleted,Age,into= c("Age","Age_When"),sep= "\\(") |>separate(Age_When,into= c("Age_When","blank"),sep= "\\)")|>select(-blank)
#Separated Age to Age and Age_When

In [5]:
Separated_Current_Rank_Points <-separate(Separated_Age_Age_When,Current_Rank,into= c("Current_Rank","Current_Rank_Points"),sep= "\\(") |>separate(Current_Rank_Points,into= c("Current_Rank_Points","blank"),sep= "\\)")|>select(-blank)
#Separated Current_Rank to Current_Rank and Current_Rank_Points

In [6]:
Best_rank_When <-separate(Separated_Current_Rank_Points,Best_rank,into= c("Best_rank","Best_rank_When"),sep= "\\(" ) |>
separate(Best_rank_When,into= c("Best_rank_When","blank"),sep= "\\)")|>
select(-blank)

In [7]:
Seperated_Current_Elo_Rank <-separate(Best_rank_When,Current_Elo_Rank,into= c("Current_Elo_Rank","Current_Elo_Rank_Points"),sep= "\\(" ) |>
separate(Current_Elo_Rank_Points,into= c("Current_Elo_Rank_Points","blank"),sep= "\\)")|>
select(-blank)

In [8]:
Seperated_Best_Elo_Rank <-separate(Seperated_Current_Elo_Rank,Best_Elo_Rank,into= c("Best_Elo_Rank","Best_Elo_Rank_Date"),sep= "\\(" ) |>
separate(Best_Elo_Rank_Date,into= c("Best_Elo_Rank_Date","blank"),sep= "\\)")|>
select(-blank)


In [9]:
Separated_Peak_Elo_Rating<-separate(Seperated_Best_Elo_Rank,Peak_Elo_Rating	,into= c("Peak_Elo_Rating","Peak_Elo_Rating_Date"),sep= "\\(" ) |>
separate(Peak_Elo_Rating_Date,into= c("Peak_Elo_Rating_Date","blank"),sep= "\\)")|>
select(-blank)

In [10]:
Seperated_GOAT_Rank <-separate(Separated_Peak_Elo_Rating,GOAT_Rank,into= c("GOAT_Rank_Points","GOAT_Rank"),sep= "\\(" ) |>
separate(GOAT_Rank,into= c("GOAT_Rank","blank"),sep= "\\)")|>
select(-blank)

In [11]:
#lacking the Surface Data to do

In [12]:
tennis_data_remain <- select(Seperated_GOAT_Rank,c(Age, Country, Current_Rank, Current_Rank_Points, Best_rank, Best_rank_When,Name))|>
 mutate(Age=as.numeric(Age),Current_Rank=as.numeric(Current_Rank),Current_Rank_Points=as.numeric(Current_Rank_Points),Best_rank=as.numeric(Best_rank)) |>
 drop_na(Current_Rank_Points)


In [13]:
tennis_data_sorted <- arrange(tennis_data_remain,desc(as.numeric(Current_Rank_Points)))
tennis_data_sorted

Age,Country,Current_Rank,Current_Rank_Points,Best_rank,Best_rank_When,Name
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
33,Spain,1,10235,1,18-08-2008,Rafael Nadal
32,Serbia,2,9720,1,04-07-2011,Novak Djokovic
38,Switzerland,3,6590,1,02-02-2004,Roger Federer
23,Russian Federation,4,5960,4,09-09-2019,Daniil Medvedev
26,Austria,5,5890,4,06-11-2017,Dominic Thiem
21,Greece,6,5375,5,05-08-2019,Stefanos Tsitsipas
22,Germany,7,3345,3,06-11-2017,Alexander Zverev
23,Italy,8,2870,8,04-11-2019,Matteo Berrettini
31,Spain,9,2630,9,04-11-2019,Roberto Bautista Agut


In [14]:
tennis_split <- initial_split(tennis_data_sorted, prop = 0.75, strata = Country)
tennis_train <- training(tennis_split)
tennis_test <- testing(tennis_split)

Warning message:
“Too little data to stratify.
• Resampling will be unstratified.”


In [15]:
tennis_data_region <- tennis_train |>
    group_by(Country) |>
    summarize(count = n()) |> 
    arrange(desc(count)) |>
    slice(1:10) 
tennis_data_region

Country,count
<chr>,<int>
France,33
United States,30
Spain,27
Italy,26
Argentina,23
Germany,20
Japan,15
Russian Federation,15
Australia,14


In [16]:
total_japan <- tennis_train |>
    filter(Country == "Japan") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1) 
total_japan

total_france <- tennis_train |>
    filter(Country == "France") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_france

total_united_states <- tennis_train |>
    filter(Country == "United States") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_united_states

total_italy <- tennis_train |>
    filter(Country == "Italy") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_italy

total_germany <- tennis_train |>
    filter(Country == "Germany") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_germany

total_spain <- tennis_train |>
    filter(Country == "Spain") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_spain

total_australia <- tennis_train |>
    filter(Country == "Australia") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_australia

total_argentina <- tennis_train |>
    filter(Country == "Argentina") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_argentina

total_russian_federation <- tennis_train |>
    filter(Country == "Russian Federation") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_russian_federation

total_brazil <- tennis_train |>
     filter(Country == "Brazil") |>
    mutate(Country_Points = sum(Current_Rank_Points)) |>
    select(Country, Country_Points) |>
    slice(1)
total_brazil


#total_country_points <- merge(total_japan, total_france, total_brazil, total_russian_federation, total_argentina, total_australia, total_spain, total_germany, total_italy, by=c("Country", "Country_Points"))

Country,Country_Points
<chr>,<dbl>
Japan,5649


Country,Country_Points
<chr>,<dbl>
France,12280


Country,Country_Points
<chr>,<dbl>
United States,11757


Country,Country_Points
<chr>,<dbl>
Italy,8434


Country,Country_Points
<chr>,<dbl>
Germany,5143


Country,Country_Points
<chr>,<dbl>
Spain,12551


Country,Country_Points
<chr>,<dbl>
Australia,7715


Country,Country_Points
<chr>,<dbl>
Argentina,6781


Country,Country_Points
<chr>,<dbl>
Russian Federation,9548


Country,Country_Points
<chr>,<dbl>
Brazil,1654


In [17]:
is.data.frame(total_japan)

[1] TRUE

In [19]:
total_country_points <- rbind(total_japan, total_france, 
                                  total_brazil, total_russian_federation, 
                                  total_argentina, total_australia,
                                  total_spain, total_germany, 
                                  total_italy, total_united_states)
total_country_points

Country,Country_Points
<chr>,<dbl>
Japan,5649
France,12280
Brazil,1654
Russian Federation,9548
Argentina,6781
Australia,7715
Spain,12551
Germany,5143
Italy,8434
